In [2]:
# MLP by pyTorch, CPU version
# Low level implementation (No Model)
# 4 class : T/C/E/L
# 3 patterns / 1 class
# 2 Layer

import numpy as np
import math as m

import torch

#from torch import tensor
#from torchvision import datasets
#from torchvision.transforms import ToTensor
import torch.nn as nn

#import torch.nn.functional as F

import torch.optim as optim

lrate = 0.01
INDIM   = 26
H1DIM    = 10
H2DIM    = 10
OUTDIM  = 4

PTTN_NUM    = 12

#x  = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
x  = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0 ],  #T-1
               [1.0, 1.0, 1.0, 1.0, 1.0, 0.0,
                     0.0, 0.0, 1.0, 0.0, 1.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0 ],  #T-2
               [1.0, 0.0, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0, 
                     0.0, 0.0, 1.0, 0.0, 0.0 ],  #T-3
               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0 ],  #C-1
               [1.0, 1.0, 1.0, 1.0, 1.0, 0.0,
                     1.0, 0.0, 0.0, 0.0, 1.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0 ],  #C-2
               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 1.0, 
                     1.0, 1.0, 1.0, 1.0, 0.0 ],  #C-3
               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0 ],  #E-1
               [1.0, 0.5, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0 ],  #E-2
               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     0.5, 1.0, 1.0, 1.0, 1.0 ],  #E-3               
               [1.0, 1.0, 0.0, 0.0, 0.0, 0.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 1.0, 1.0, 1.0, 1.0 ],  #L-1
               [1.0, 1.0, 0.0, 0.0, 0.0, 0.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 1.0, 
                     1.0, 1.0, 1.0, 1.0, 0.0 ],  #L-2
               [1.0, 1.0, 0.0, 0.0, 0.0, 0.0,
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     1.0, 0.0, 0.0, 0.0, 0.0, 
                     0.3, 1.0, 1.0, 1.0, 1.0 ] ] )  #L-3                            
                              
#t  = np.array([ [1.0, 0.0, 0.0, 0.0],
yt  = torch.tensor([ [1.0, 0.0, 0.0, 0.0],               
                [1.0, 0.0, 0.0, 0.0],
                [1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0, 1.0] ])




In [ ]:
print(x.shape)
print(yt.shape)

In [ ]:
### 1 Layer MLP ==> SLP, no sigmoid activation
### variables & gradient setting
w = torch.randn(INDIM, OUTDIM, requires_grad=True)
b = torch.randn(OUTDIM, requires_grad=True)

### Loss & Optimizer
#loss = torch.nn.functional.binary_cross_entropy_with_logits(z, yt)
floss = nn.MSELoss()
optimizer = optim.SGD([w, b], lr=lrate)

sigmoid = nn.Sigmoid()

###############################
### Test 1 epoch
'''
optimizer.zero_grad()
z = torch.matmul(x, w)+b
#print(x.shape, w.shape, z.shape)
loss = floss(z, yt)
loss.backward()
optimizer.step()
'''
###############################
for epoch in range(20000) :
  optimizer.zero_grad()
  zt = torch.matmul(x, w)+b
  z = sigmoid(zt)

  loss = floss(z, yt)
  loss.backward()
  optimizer.step()
  if epoch%1000 == 0 :
    print("Epoch:", epoch)
    print("z:", z)


In [ ]:
### 1 Layer MLP ==> SLP
### variables & gradient setting
W1 = torch.randn(INDIM, OUTDIM, requires_grad=True)
B1 = torch.randn(OUTDIM, requires_grad=True)

### Loss & Optimizer
#loss = torch.nn.functional.binary_cross_entropy_with_logits(z, yt)
floss = nn.MSELoss()
optimizer = optim.SGD([W1, B1], lr=lrate)

sigmoid = nn.Sigmoid()

###############################
for epoch in range(20000) :
  optimizer.zero_grad()
  
  zt = torch.matmul(x, W1)+B1
  z = sigmoid(zt)

  loss = floss(z, yt)
  loss.backward()
  optimizer.step()

  if epoch%1000 == 0 :
    print("Epoch:", epoch)
    print("z:", z)


In [ ]:
### 2 Layer MLP
### variables & gradient setting
W1 = torch.randn(INDIM, H1DIM, requires_grad=True)
B1 = torch.randn(H1DIM, requires_grad=True)
W2 = torch.randn(H1DIM, OUTDIM, requires_grad=True)
B2 = torch.randn(OUTDIM, requires_grad=True)

### Loss & Optimizer
#loss = torch.nn.functional.binary_cross_entropy_with_logits(z, yt)
floss = nn.MSELoss()
optimizer = optim.SGD([W1, B1, W2, B2], lr=lrate)

sigmoid = nn.Sigmoid()

###############################
for epoch in range(20000) :
  optimizer.zero_grad()

  z1t = torch.matmul(x, W1)+B1
  z1 = sigmoid(z1t)
  z2t = torch.matmul(z1, W2)+B2
  z2 = sigmoid(z2t)

  loss = floss(z2, yt)
  loss.backward()
  optimizer.step()

  if epoch%1000 == 0 :
    print("Epoch:", epoch)
    print("z:", z2)


In [ ]:
### 3 Layer MLP
### variables & gradient setting
W1 = torch.randn(INDIM, H1DIM, requires_grad=True)
B1 = torch.randn(H1DIM, requires_grad=True)
W2 = torch.randn(H1DIM, H2DIM, requires_grad=True)
B2 = torch.randn(H2DIM, requires_grad=True)
W3 = torch.randn(H2DIM, OUTDIM, requires_grad=True)
B3 = torch.randn(OUTDIM, requires_grad=True)

### Loss & Optimizer
#loss = torch.nn.functional.binary_cross_entropy_with_logits(z, yt)
floss = nn.MSELoss()
optimizer = optim.SGD([W1, B1, W2, B2, W3, B3], lr=lrate)

sigmoid = nn.Sigmoid()

###############################
for epoch in range(20000) :
  optimizer.zero_grad()

  z1t = torch.matmul(x, W1)+B1
  z1 = sigmoid(z1t)
  z2t = torch.matmul(z1, W2)+B2
  z2 = sigmoid(z2t)
  z3t = torch.matmul(z2, W3)+B3
  z3 = sigmoid(z3t)

  loss = floss(z3, yt)
  loss.backward()
  optimizer.step()

  if epoch%1000 == 0 :
    print("Epoch:", epoch)
    print("z:", z3)
